In [ ]:
###### print("Please read:")

print("")

print("we wish to record your response data")

print("to an anonymised public data repository. ")

print("Your data will be used for educational teaching purposes")

print("practising data analysis and visualisation.")

print("")

print("Please type yes in the box below if you consent to the upload.")

result = input("> ")

if result == "yes":

    print("Thanks - your data will be uploaded.")

#send_to_google_form(data_dict, form_url)

else:

    print("No problem we hope you enjoyed the test.")

### import requests
from bs4 import BeautifulSoup
import json
import time
from IPython.display import display, HTML, Image, clear_output
import requests

def send_to_google_form(data_dict, form_url):
    ''' Helper function to upload information to a corresponding Google Form 
        You are not expected to follow the code within this function!
    '''
    form_id = form_url[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')

    if content is None:
        print("Script tag not found. Data not uploaded.")
        return False

    content_text = content.text[27:-1] if content.text else None

    if not content_text:
        print("Content text not found. Data not uploaded.")
        return False

    try:
        result = json.loads(content_text)[1][1]
        print("Content of the script tag:")
        print(result)
        form_dict = {}
    
        loaded_all = True
        for item in result:
            # Remove non-alphanumeric characters from the item name
            item_name_cleaned = ''.join(char.lower() for char in item[1] if char.isalnum())
            
            # Check if the cleaned item name matches any key in the data dictionary
            matching_keys = [key for key in data_dict.keys() if item_name_cleaned in ''.join(char.lower() for char in key if char.isalnum())]
            
            if not matching_keys:
                print(f"Form item {item[1]} not found. Data not uploaded.")
                loaded_all = False
                return False
            
            # Use the first matching key as the data dictionary key
            form_dict[f'entry.{item[4][0][0]}'] = data_dict[matching_keys[0]]
        
        post_result = requests.post(post_form_url, data=form_dict)
        return post_result.ok
    
    except (json.JSONDecodeError, IndexError) as e:
        print(f"Error decoding JSON or accessing result: {e}")
        return False
            
# Reaction time test
print("We are going to test your memory!")

print("Please enter your name:")
ans1 = input("> ")

print("Please enter your age:")
ans2 = input("> ")

print("Please enter your gender:")
ans3 = input("> ")

results = []
form_url = "https://docs.google.com/forms/d/e/1FAIpQLSdWWMvaCnN_fZPOb2fV5T-DiBzvv9IEpwuo7kM4SsCs1RGOaQ/viewform?usp=sf_link"

# Function to display a grid and wait for 5 seconds
def display_grid(grid):
    display(grid)
    time.sleep(5)
    clear_output(wait=False)

# Function to ask a question and return the user's answer
def ask_question(question, correct_answer):
    user_answer = input(question)
    if user_answer == correct_answer:
        print("Correct!")
        return 1
    else:
        print(f"The correct answer is {correct_answer}.")
        return 0

# Welcome message
welcome_msg = HTML("<span style='font-size:27px'> Welcome to the Memory test</span>")
display(welcome_msg)
print("Each grid will be shown for 10 seconds, try to remember, good luck!")

score = 0
start_time = time.time()

# Define the grids
Grid_1 = Image("Grid_1.png", width = 400)
Grid_2 = Image("Grid_2.png", width = 400)
Grid_3 = Image("Grid_3.png", width = 400)
Grid_4 = Image("Grid_4.png", width = 400)
Grid_5 = Image("Grid_5.png", width = 400)

# Questions and answers
questions_and_answers = [
    (Grid_1, [
        ("How many circles are there in this grid?", "5"),
        ("What shape was on the lower left corner?", "circle")
    ]),
    (Grid_2, [
        ("How many objects were in green?", "4"),
        ("How many red circles were there?", "2")
    ]),
    (Grid_3, [
        ("How many triangles were there?", "3"),
        ("What is the color of the square in the middle?", "green"),
        ("How many circles were colored yellow?", "0")
    ]),
    (Grid_4, [
        ("How many types of shapes were there?", "4"),
        ("What shape was on the middle upper grid?", "triangle"),
        ("How many red squares were there?", "2"),
        ("What is the color of the shape located at upper right corner?", "red")
    ]),
    (Grid_5, [
        ("Which shape was colored yellow?", "kite")
    ])
]

for grid, qa_set in questions_and_answers:
    display_grid(grid)
    for question, correct_answer in qa_set:
        score += ask_question(question, correct_answer)

end_time = time.time()
time_taken = end_time - start_time

print("Well done!")
time.sleep(1)
print("The score you receive is:", score)

if score <= 2:
    print("You might need a GP. Immediately.")

print(f"You took {time_taken:.2f} seconds")

# Upload the result to the Google Form
upload_success = send_to_google_form({'Name': ans1, 'Age': ans2, 'Gender': ans3, 'Score': score, 'Time taken': time_taken},form_url)

# Print the result
if upload_success:
    print("Data uploaded successfully!")
else:
    print("Data upload failed.")